In [1]:
import OpenDartReader
from pykrx import stock
import pandas as pd
from datetime import datetime
import time
import pickle
from tqdm import tqdm

In [2]:
crtfc_key = open('crtfc_key.txt',mode='r').readline()
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'

In [3]:
date = datetime(2022,5,30)
kospi_list1 = stock.get_market_ticker_list(date, market='KOSPI')
kosdaq_list1 = stock.get_market_ticker_list(date, market='KOSDAQ')
kospi_list2 = list(stock.get_market_cap_by_ticker(date=date, market='KOSPI').index)
kosdaq_list2 = list(stock.get_market_cap_by_ticker(date=date, market='KOSDAQ').index)

In [4]:
kospi_list = list(set(kospi_list1) | set(kospi_list2))
kosdaq_list = list(set(kosdaq_list1) | set(kosdaq_list2))

In [5]:
print(len(kosdaq_list))
print(len(kospi_list))

1571
946


In [6]:
dart = OpenDartReader(crtfc_key)

In [ ]:
1분기보고서 : 11013
반기보고서 : 11012
3분기보고서 : 11014
사업보고서 : 11011
fs_div	개별/연결구분	STRING(3)	Y	CFS:연결재무제표, OFS:재무제표

In [9]:
year = 2022
reprt = '03'
if reprt == '03':
    reprt_code = 11013
elif reprt == '06':
    reprt_code = 11012
elif reprt == '09':
    reprt_code = 11014
elif reprt == '12':
    reprt_code = 11011
else:
    raise ValueError('reprt 확인!!!')

resultTotal = pd.DataFrame()
listError = []
listStock = kospi_list+kosdaq_list
timeStart = datetime.now()
for stockCode in tqdm(listStock):
    time.sleep(0.1)
    if (listStock.index(stockCode)%500 == 0) & (listStock.index(stockCode) != 0):
        timeEnd = datetime.now()
        if (timeEnd-timeStart).seconds < 60:
            time.sleep(int(60-(timeEnd-timeStart).seconds))
        timeStart = datetime.now()
    result = dart.finstate_all(stockCode, year, reprt_code=reprt_code,fs_div='CFS')
    if isinstance(result,type(None)):
        result = dart.finstate_all(stockCode, year, reprt_code=reprt_code,fs_div='OFS')
    if isinstance(result,type(None)):
        listError.append(stockCode)
    else:
        result['stock_code'] = stockCode
        resultTotal = pd.concat([resultTotal,result],axis=0)
resultTotal.reset_index(drop=True,inplace=True)        
with open(f'Raw_Finance/dartAPI/{year}{reprt}.pickle','wb') as handle:
    pickle.dump(resultTotal, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(f'Raw_Finance/dartAPI/{year}{reprt}_error.pickle','wb') as handle:
    pickle.dump(listError, handle, protocol=pickle.HIGHEST_PROTOCOL)

100%|███████████████████████████████████████████████████████| 2517/2517 [18:07<00:00,  2.31it/s]


In [10]:
len(listError)

337

In [11]:
listError

['002785',
 '357120',
 '003690',
 '003470',
 '003610',
 '105560',
 '030210',
 '37550K',
 '010050',
 '377190',
 '003545',
 '33626K',
 '032830',
 '005305',
 '039490',
 '33637K',
 '152550',
 '030610',
 '145995',
 '38380K',
 '000370',
 '055550',
 '00279K',
 '000157',
 '008560',
 '004985',
 '37550L',
 '900140',
 '350520',
 '003925',
 '002995',
 '000400',
 '090355',
 '395400',
 '404990',
 '001080',
 '139130',
 '000540',
 '402340',
 '026890',
 '007630',
 '016360',
 '034830',
 '003465',
 '005257',
 '011155',
 '003540',
 '029780',
 '00806K',
 '071055',
 '004545',
 '000547',
 '084695',
 '175330',
 '019175',
 '001750',
 '004255',
 '088980',
 '009155',
 '088350',
 '019685',
 '085620',
 '18064K',
 '00104K',
 '003475',
 '000885',
 '380440',
 '001465',
 '000215',
 '001725',
 '005389',
 '002787',
 '365550',
 '071050',
 '002795',
 '006805',
 '000810',
 '024110',
 '016385',
 '001065',
 '007575',
 '096300',
 '417310',
 '005725',
 '28513K',
 '007815',
 '002630',
 '001045',
 '004415',
 '090435',
 '000140',

In [20]:
cases.isnull().sum()

rcept_no                 0
reprt_code               0
bsns_year                0
corp_code                0
sj_div                   0
sj_nm                    0
account_id               0
account_nm               0
account_detail           0
thstrm_nm                0
thstrm_amount            0
frmtrm_nm            68247
frmtrm_amount        68247
ord                      0
currency                 0
thstrm_add_amount    80577
frmtrm_q_nm          28939
frmtrm_q_amount      32325
frmtrm_add_amount    80589
stock_code               0
dtype: int64

In [19]:
import pickle
with open('Raw_Finance/dartAPI/202203.pickle','rb') as handle:
    cases = pickle.load(handle)
cases

,rcept_no,reprt_code,bsns_year,corp_code,sj_div,sj_nm,account_id,account_nm,account_detail,thstrm_nm,thstrm_amount,frmtrm_nm,frmtrm_amount,ord,currency,thstrm_add_amount,frmtrm_q_nm,frmtrm_q_amount,frmtrm_add_amount,stock_code
0,20220513001346,11013,2022,00144818,BS,재무상태표,ifrs-full_CurrentAssets,유동자산,-,제 83 기 1분기말,100846644417,제 82 기말,98887921935,1,KRW,NaN,NaN,NaN,NaN,000220
1,20220513001346,11013,2022,00144818,BS,재무상태표,ifrs-full_CashAndCashEquivalents,현금및현금성자산,-,제 83 기 1분기말,49313966791,제 82 기말,50263467930,2,KRW,NaN,NaN,NaN,NaN,000220
2,20220513001346,11013,2022,00144818,BS,재무상태표,ifrs-full_TradeAndOtherCurrentReceivables,매출채권 및 기타유동채권,-,제 83 기 1분기말,26361175495,제 82 기말,22793030277,3,KRW,NaN,NaN,NaN,NaN,000220
3,20220513001346,11013,2022,00144818,BS,재무상태표,ifrs-full_Inventories,재고자산,-,제 83 기 1분기말,24756353921,제 82 기말,25225283637,5,KRW,NaN,NaN,NaN,NaN,000220
4,20220513001346,11013,2022,00144818,BS,재무상태표,dart_OtherCurrentAssets,기타유동자산,-,제 83 기 1분기말,194963082,제 82 기말,393628120,6,KRW,NaN,NaN,NaN,NaN,000220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97134,20220513001236,11013,2022,00163318,SCE,자본변동표,ifrs-full_Equity,기말자본,자본 [member]|지배기업의 소유주에게 귀속되는 자본 [member]|기타자본구...,제 53 기 1분기,954638613,NaN,NaN,3,KRW,NaN,제 52 기 1분기,954638613,NaN,006920
97135,20220513001236,11013,2022,00163318,SCE,자본변동표,ifrs-full_Equity,기말자본,자본 [member]|지배기업의 소유주에게 귀속되는 자본 [member]|기타포괄손...,제 53 기 1분기,-618987110,NaN,NaN,3,KRW,NaN,제 52 기 1분기,-622900837,NaN,006920
97136,20220513001236,11013,2022,00163318,SCE,자본변동표,ifrs-full_Equity,기말자본,자본 [member]|지배기업의 소유주에게 귀속되는 자본 [member]|이익잉여금...,제 53 기 1분기,12540114414,NaN,NaN,3,KRW,NaN,제 52 기 1분기,12858309107,NaN,006920
97137,20220513001236,11013,2022,00163318,SCE,자본변동표,ifrs-full_Equity,기말자본,자본 [member]|지배기업의 소유주에게 귀속되는 자본 [member]|자본금 [...,제 53 기 1분기,5460000000,NaN,NaN,3,KRW,NaN,제 52 기 1분기,5460000000,NaN,006920
